In [1]:
import cv2
import os
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision



path = 'Yoga poses'
class ExitLoopException(Exception):
    pass

In [2]:
def draw_landmarks_on_image(rgb_image, detection_result):
    pose_landmarks_list = detection_result.pose_landmarks
    annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
    for idx in range(len(pose_landmarks_list)):
        pose_landmarks = pose_landmarks_list[idx]
    
        # Draw the pose landmarks.
        pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        pose_landmarks_proto.landmark.extend([
          landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
          annotated_image,
          pose_landmarks_proto,
          solutions.pose.POSE_CONNECTIONS,
          solutions.drawing_styles.get_default_pose_landmarks_style())
    return annotated_image

In [3]:
def findAngle(X1:np.array, X2:np.array):
    theta=np.arccos((np.dot(X1,X2)/(np.linalg.norm(X1)*np.linalg.norm(X2))))
    if theta<0:
        return theta+360
    return theta

In [4]:
def calculate_accuracy(x1,x2,tolerance=10):
    normalized_tolerance = tolerance / 360.0
    sums=0
    for key in x1.keys():
        absolute_difference = abs(x1[key] -x2[key])
        sums+=absolute_difference
    
    if (sums/len(x1)) <= normalized_tolerance:
        return 1 
    else:
        return 0 
    

In [5]:
def angle_btw(detection_result):
    angle_btw={}
    left_shoulder=[]
    right_shoulder=[]
    left_hip=[]
    right_hip=[]
    left_elbow=[]
    right_elbow=[]
    left_wrist=[]
    right_wrist=[]
    left_ankle=[]
    right_ankle=[]
    left_knee=[]
    right_knee=[]
    for position in detection_result.pose_landmarks:
        left_shoulder.append(position[11].x)
        left_shoulder.append(position[11].y)
        left_shoulder.append(position[11].z)
        
        right_shoulder.append(position[12].x)
        right_shoulder.append(position[12].y)
        right_shoulder.append(position[12].z)
        
        
        left_hip.append(position[23].x)
        left_hip.append(position[23].y)
        left_hip.append(position[23].z)
        
        right_hip.append(position[24].x)
        right_hip.append(position[24].y)
        right_hip.append(position[24].z)
        
        
        left_elbow.append(position[13].x)
        left_elbow.append(position[13].y)
        left_elbow.append(position[13].z)
        
        right_elbow.append(position[14].x)
        right_elbow.append(position[14].y)
        right_elbow.append(position[14].z)
        
        left_wrist.append(position[15].x)
        left_wrist.append(position[15].y)
        left_wrist.append(position[15].z)
        
        right_wrist.append(position[16].x)
        right_wrist.append(position[16].y)
        right_wrist.append(position[16].z)
        
        left_knee.append(position[25].x)
        left_knee.append(position[25].y)
        left_knee.append(position[25].z)
        
        right_knee.append(position[26].x)
        right_knee.append(position[26].y)
        right_knee.append(position[26].z)
        
        
        left_ankle.append(position[27].x)
        left_ankle.append(position[27].y)
        left_ankle.append(position[27].z )
        
        right_ankle.append(position[28].x)
        right_ankle.append(position[28].y)
        right_ankle.append(position[28].z)

    angle_btw['angle btw left shoulder and elbow']=findAngle(left_shoulder,left_elbow) 
    angle_btw['angle btw right shoulder and elbow']=findAngle(right_shoulder,right_elbow) 
    angle_btw['angle btw left shoulder and hip']=findAngle(left_shoulder,left_hip) 
    angle_btw['angle btw right shoulder and hip']=findAngle(right_shoulder,right_hip)  
    angle_btw['angle btw left wrist and elbow']=findAngle(right_wrist,right_elbow)
    angle_btw['angle btw right wrist and elbow']=findAngle(right_wrist,right_elbow)
    angle_btw['angle btw left knee and hip']=findAngle(left_knee,left_hip) 
    angle_btw['angle btw right knee and hip']=findAngle(right_knee,right_hip)
    angle_btw['angle btw left knee and ankle']=findAngle(left_shoulder,left_ankle) 
    angle_btw['angle btw right knee and ankle']=findAngle(right_shoulder,right_ankle)
    return angle_btw

In [6]:
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

correct_angle={}

try:
    for file in os.listdir(path):
        new_path=os.path.join(path, file)
        for image in os.listdir(new_path):
            print(image)
            image_name=image
            image = mp.Image.create_from_file(os.path.join(new_path, image))
            detection_result = detector.detect(image)
            image = cv2.cvtColor(image.numpy_view(), cv2.COLOR_RGB2BGR)
            annotated_image = draw_landmarks_on_image(image, detection_result)
            bgr_image = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)
            cv2.imshow('Annotated Image', bgr_image)
            cv2.waitKey(100)
            correct_angle[f"{image_name}"]=angle_btw(detection_result)
except Exception as e:
    print(e)
finally:
    cv2.destroyAllWindows()

Adho Mukha Svanasana (Advance).jpeg
Adho Mukha Svanasana (Beginners).jpeg
Adho Mukha Svanasana (intermediate).jpeg
Ananda Balasana.jpeg
Ardha Matsyendrasana.jpg
Bhujangasana(advance).jpeg
Bhujangasana(beginner).jpeg
Bhujangasana(intermediate).jpeg
Parsvottanasana(advance).jpg
Parsvottanasana(beginner).jpg
Parsvottanasana(intermediate).jpg
Paschimottanasana.jpg
Utthita Parvakonasana(advance).jpeg
Utthita Parvakonasana(beginner).jpeg
Utthita Parvakonasana(intermediate).jpeg
Virabhadrasana I (advance).jpeg
Virabhadrasana I (beginner).jpeg
Virabhadrasana I(intermediate).jpeg
Virabhadrasana II(advance).jpeg
Virabhadrasana II(beginner).jpeg
Virabhadrasana II(intermediate).jpeg
Virabhadrasana III.jpg


In [7]:
def main ():
    cap=cv2.VideoCapture(0)
    

    try:
        while 1:
            for file in os.listdir(path):
                new_path=os.path.join(path, file)
                for image in os.listdir(new_path):
                    image_name=image
                    overlay_image =cv2.imread(os.path.join(new_path, image))
                    print(image_name)
                    move=False
                    while not move :
                        ret, frame = cap.read()
                        frame = cv2.flip(frame, 1)
                        framergb= cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        framergb = framergb.astype(np.uint8)
                        overlay_image_resized = cv2.resize(overlay_image, (frame.shape[1] // 4, frame.shape[0] // 4))
                        frame[:overlay_image_resized.shape[0], :overlay_image_resized.shape[1]] = overlay_image_resized
                        image=mp.Image(image_format=mp.ImageFormat.SRGB, data=framergb)
                        detection_result = detector.detect(image)
                        angle=angle_btw(detection_result)
                        annotated_image = draw_landmarks_on_image(frame, detection_result)
                        cv2.imshow("Webcam with Overlay", annotated_image)
                        if cv2.waitKey(1)==ord('q'):
                            raise ExitLoopException("End")
                        if calculate_accuracy(angle,correct_angle[f"{image_name}"]):
                            move=True
                    
            if cv2.waitKey(1) == ord('q'):
                break
            
    except Exception as e:
        print(e)
    finally:
        cap.release()
        cv2.destroyAllWindows()

In [8]:
main()

Adho Mukha Svanasana (Advance).jpeg


C:\Users\Ayush Aditya\AppData\Local\Temp\ipykernel_20364\2709731559.py:2: RuntimeWarning: invalid value encountered in scalar divide
  theta=np.arccos((np.dot(X1,X2)/(np.linalg.norm(X1)*np.linalg.norm(X2))))


End
